In [8]:
#james chartouni
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler

In [16]:
training_data = pd.read_csv("cleaned_input/train_consolidated.csv")
training_data = training_data.drop(['msno'], axis=1)

Index(['Unnamed: 0', 'msno', 'is_churn', 'trans_count', 'logs_count',
       'Unnamed: 0.1', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date', 'registration_init_year',
       'registration_init_month', 'registration_init_date',
       'expiration_date_year', 'expiration_date_month', 'expiration_date_date',
       'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew', 'transaction_date',
       'membership_expire_date', 'is_cancel', 'date', 'num_25', 'num_50',
       'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs', 'discount',
       'is_discount', 'amt_per_day', 'membership_duration',
       'membership_duration_M', 'registration_duration',
       'registration_duration_M', 'reg_mem_duration', 'reg_mem_duration_M',
       'notAutorenew_&_cancel', 'long_time_user'],
      dtype='object')


In [24]:
#split data 
y = training_data["is_churn"].values
X = training_data.drop(["is_churn"], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, )

In [18]:
#logistic regression 

logistic_pipe = make_pipeline(LogisticRegression())
score = cross_val_score(logistic_pipe, X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



ValueError: could not convert string to float: '2017-01-27'

In [23]:
#Light GBM 
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# generate a feature name
feature_name = X_train.columns

print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_train,  # eval training data
              )

# save model to file
gbm.save_model('model.txt')

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))


Start training...


ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields msno, gender, registration_init_time, expiration_date, transaction_date, membership_expire_date